# Imports

In [ ]:
import logging
from bs4 import BeautifulSoup, Tag

In [ ]:
from core.parser import rockwell_l5x_parser

# Setup

In [ ]:
project_path = r"path\to\project.L5X"

# Load project
https://literature.rockwellautomation.com/idc/groups/literature/documents/rm/1756-rm014_-en-p.pdf

In [ ]:
def view_attrs(objs: list[Tag]):
    for o in objs:
        print(o.attrs)


def ellipsis(text):
    print(str(text)[:80] + "...")

In [ ]:
with open(project_path, "r", encoding="utf-8") as f:
    project_soup = BeautifulSoup(f, features="xml")

- `soup.RSLogix5000Content`
- `find_all("Program")`
- `find_all("Routine")`
- `find_all("Rung")`
- `find_all("Text")`

In [ ]:
# view_attrs(project_soup.find("Tasks").find_all("Task"))
# view_attrs(project_soup.find("Programs").find_all("Program"))
# project_soup.find("Programs").find_next("Program").find("Routines").find_next("Routine")

# Single routine processing

In [ ]:
routine_xml = (
    project_soup.find("Programs")
    .find_next("Program")
    .find("Routines")
    .find("Routine", attrs={"Name": "<ROUTINE NAME>"})
)
ellipsis(routine_xml)

routine_plain = ""
for rung in routine_xml.find_all("Rung"):
    routine_plain += rung.find("Text").text
ellipsis(routine_plain)

routine = rockwell_l5x_parser.parse(routine_plain)
ellipsis(routine.pretty())

## Dependency tree for OTE (latch)
Walk AST and look for OTE instructions. Record all tags untill some OTE is met - those tags are dependencies of tag in OTE instruction. If no OTE met discard recorded tags.

In [ ]:
from treelib.exceptions import DuplicatedNodeIdError
from treelib.tree import Tree
from treelib.node import Node
from lark import Tree as LarkTree, Token as LarkToken
from collections import defaultdict
from hashlib import sha256
from uuid import uuid4

In [ ]:
def _node_id(tag: str) -> str:
    return sha256(tag.encode()).hexdigest()


def _build_tag_reference(t: LarkTree):
    tag = t.children[0].value

    for st in t.iter_subtrees():
        if st == t:
            continue

        if isinstance(st.children[0], LarkTree):
            val = _build_tag_reference(st.children[0])
        else:
            val = st.children[0].value

        if st.data == "index_access":
            val = "[" + val + "]"
        if st.data == "dot_member_access":
            val = "." + val
        if st.data == "colon_module_access":
            val = "." + val

        tag = tag + val
    return tag

In [ ]:
def build_direct_dependecies(routine: LarkTree):
    # Look for OTE instruction and mark all previously met tags as dependencies

    forest: dict[str, Tree] = {}
    for rg in routine.children:
        # Ignore NOP comment rungs
        if (
            len(rg.children) == 1
            and isinstance(rg.children[0], LarkTree)
            and isinstance(rg.children[0].children[0], LarkToken)
            and rg.children[0].children[0].value == "NOP"
        ):
            continue
        # print(rg)

        dependencies: set[Node] = set()  # List of tags
        targets: set[Node] = set()  # List of tags
        for tr in rg.iter_subtrees():
            if tr.data == "instruction":
                # print(tr)

                _dest = targets if tr.children[0] == "OTE" else dependencies
                for tn in tr.iter_subtrees():
                    if tn.data == "tag_reference":
                        opcode = tr.children[0].value
                        tag = _build_tag_reference(tn)

                        n = Node(
                            tag, _node_id(tag), data={"opcode": opcode, "tag": tag}
                        )

                        # print(opcode)
                        # print(tn)
                        # print(tag)

                        _dest.add(n)
        # print(dependencies, targets)

        # Convert plain text tags to nodes
        for t in targets:
            tree = Tree()
            tree.add_node(t)
            for d in dependencies:
                try:
                    tree.add_node(d, t)
                except DuplicatedNodeIdError:
                    # TODO: Self dependencies are not supported
                    continue

            forest[t.tag] = tree
            # tree.show()

    return forest

In [ ]:
direct_dependencies = build_direct_dependecies(routine)

In [ ]:
def expand_dependency_level(root: Tree, dd: dict[str, Tree]):
    for leave in root.leaves():
        try:
            sub_tree = dd[leave.tag]
        except KeyError:
            continue

        predecessor = leave.predecessor(root.identifier)

        # print(leave.identifier)
        # print(predecessor)
        # print(sub_tree)

        root.remove_node(leave.identifier)
        try:
            root.paste(predecessor, sub_tree)
        except ValueError:
            continue

    return Tree(root, deep=True)


explore = Tree(direct_dependencies["<TAG NAME>"], deep=True)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore = expand_dependency_level(explore, direct_dependencies)
explore.show()

In [ ]:
routines = []
for routine_xml in project_soup.find_all("Routine"):
    plain = ""
    for rung in routine_xml.find_all("Rung"):
        plain += rung.find("Text").text
    if not text:
        continue
    try:
        routines.append(rockwell_l5x_parser.parse(plain))
    except Exception:
        print("Parser bug encoutered")
        print(plain)
        continue

In [ ]:
for routine in routines:
    dd = build_direct_dependecies(routine)
    direct_dependencies.update(dd)

In [ ]:
direct_dependencies